In [148]:
import requests
from bs4 import BeautifulSoup

In [149]:
base_url = 'http://books.toscrape.com'

In [150]:
def get_url_of_nth_page(n):
    return (base_url + '/catalogue/page-' + str(n + 1) + '.html')

In [151]:
def return_response_from_page_index(n):
    
    url = get_url_of_nth_page(n)
    response = requests.get(url)
    
    return response

In [152]:
def return_img_src_from_book_tag(book_tag):
    
    attrs_dict = {
        'class': 'image_container'
    }
    
    img_container_tag = book_tag.find('div', attrs_dict)
    img_tag = img_container_tag.find('img')
    
    src = img_tag['src']
    
    return src

In [213]:
def return_book_name_from_book_tag(book_tag):
    
    h3_tag = book_tag.find('h3')
    
    anchor_tag = h3_tag.find('a')
    
    name = anchor_tag['title'].replace('â\x80\x99', chr(39)).replace('â\x80\x9c', chr(34))
    
    name = name.replace('"', (chr(34) + chr(34)))
    
    if not name.find(',') == -1:
        name = '"' + name + '"'
    
    return name

In [165]:
def return_book_price_from_book_tag(book_tag):
    
    attrs_dict = {
        'class': 'product_price'
    }
    
    price_container_tag = book_tag.find('div', attrs_dict)
    
    attrs_dict = {
        'class': 'price_color'
    }
    
    price_tag = price_container_tag.find('p', attrs_dict)
    
    price = price_tag.text[1:]
    
    return price

In [196]:
def return_data_from_html_text(cntnt):
    parsed_html = BeautifulSoup(cntnt, 'html.parser', from_encoding = "utf-8")
    
    books_tag = parsed_html.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    
    thumbnails_src = [return_img_src_from_book_tag(book) for book in books_tag]
    
    names_txt = [return_book_name_from_book_tag(book) for book in books_tag]
    
    prices = [return_book_price_from_book_tag(book) for book in books_tag]
    
    return_tuple = (thumbnails_src, names_txt, prices)
        
    return return_tuple

In [197]:
all_thumbnail_srcs = []
all_book_names = []
all_book_prices = []

In [214]:
for page_index in range(50):
    
    response = return_response_from_page_index(page_index)
    parsed_tuple = return_data_from_html_text(response.content)
    
    all_thumbnail_srcs += parsed_tuple[0]
    all_book_names += parsed_tuple[1]
    all_book_prices += parsed_tuple[2]

In [215]:
# A CSV file with a structure similar to this was needed to submitted as the result

with open('result.csv', 'w', encoding='utf-8') as file:
    file.write('image_url,book_title,product_price\n')
    for i in range(len(all_thumbnail_srcs)):
        try:
            file.write(all_thumbnail_srcs[i])
            file.write(',')
            
            file.write(all_book_names[i])
            
            file.write(',')
            
            file.write('£')
            file.write(all_book_prices[i])
            
            file.write('\n')
        except:
            print('problem i = ',i)
            break